# Week 9: Sentence Level Classification with BERT

Your goal this week is to train a classifier that can predict the CEFR level of any given sentence. In this notebook we will guide you through the process of using 🤗[Hugging Face](https://huggingface.co/) and its transformers library as the training framework, with [Pytorch](https://pytorch.org/) as the deep learning backend, but feel free to use [TensorFlow](https://www.tensorflow.org) if that's what you are more familiar with.

For this assignment we will provide a dataset containing sentences with the corresponding CEFR level, and you have to use BERT and train a sentence classifier with this dataset.

## Prepare your environment

As always, we highly recommend that you install all packages with a virtual environment manager, like [venv](https://packaging.python.org/en/latest/guides/installing-using-pip-and-virtual-environments/) or [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html), to prevent version conflicts of different packages.  

### Install CUDA
Deep learning is a computionally extensive process. It takes lots of time if relying only on the CPU, especially when it's trained on a large dataset. That's why using GPU instead is generally recommended.  
To use GPU for computation, you have to install [CUDA toolkit](https://developer.nvidia.com/cuda-toolkit) as well as the [cuDNN library](https://developer.nvidia.com/cudnn) provided by NVIDIA.  

If you already had CUDA installed on your machine, then great! You're done here.  
If you don't, you can refer to [Appendix](#Appendix-1-Install-CUDA) to see how to do so.


### Install python packages
The following python packages will be used in this tutorial:

1. `numpy`: for matrix operation
2. `scikit-learn`: for label encoding
3. `datasets`: for data preparation
4. `transformers`: for model loading and finetuing
5. `pytorch`: the backend DL framework
  - Note that the pt version must support the CUDA version you've installed if you want to use GPU.

### Select GPU(s) for your backend

Skip this section if you have no intension of using GPU with tensorflow/pytorch.

In [1]:
import os

# select your GPU. Note that this should be set before you load tensorflow or pytorch.
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# To use multiple GPUs, combine all GPU ID with commas
# e.g. >>> os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,3'

In [2]:
import torch

# Check if any GPU is used
torch.cuda.is_available()

/home/nlplab/yhc/env/nlp_hw/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Prepare the dataset

Before starting the training, we need to load and process our dataset - but wait, let's decide which model we want to use first.  

In the highly unlikely chance you've never heard of it, [BERT](https://arxiv.org/abs/1810.04805) (**B**idirectional **E**ncoder **R**epresentations from **T**ransformers) is a language model proposed by Google AI in 2018, and it's currently one of the most popular models used in NLP.  
You can learn more about it here:
- [BERT Explained: A Complete Guide with Theory and Tutorial](https://towardsml.com/2019/09/17/bert-explained-a-complete-guide-with-theory-and-tutorial/) by Samia, 2019.


However, we will not directly use BERT in this tutorial, because it's large and takes too long to train. Instead, we'll be using [DistilBert](https://medium.com/huggingface/distilbert-8cf3380435b5), a version of BERT that while light-weight, reserves 95% of its original accuracy.




In [3]:
# the model you want to use. Available models can be found here: https://huggingface.co/models
MODEL_NAME = 'distilbert-base-uncased'

### Load data

Similar to the `transformers` library, `datasets` is also a package by huggingface. It contains many public datasets online and can help us with the data processing.  
We can use `load_dataset` function to read the input `.csv` file provided for this assignment.

Reference:
 - [Official datasets document](https://huggingface.co/docs/datasets)
 - [datasets.load_dataset](https://huggingface.co/docs/datasets/loading.html)

In [4]:
#! unzip data -d

In [5]:
# [ TODO ] load the data using the load_dataset function
from datasets import load_dataset
data_files = {"train": "evp.train.csv", "test": "evp.test.csv"}
dataset = load_dataset("data", data_files=data_files)


Using custom data configuration data-3a5053b2d0593831
Found cached dataset csv (/home/nlplab/yhc/.cache/huggingface/datasets/csv/data-3a5053b2d0593831/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 451.70it/s]


In [6]:
print(dataset['train'])
print(dataset['train'][1])
print(dataset['train']['text'][:5])

Dataset({
    features: ['text', 'level'],
    num_rows: 20720
})
{'text': 'You can contact me by e-mail.', 'level': 'A1'}
['My mother is having her car repaired.', 'You can contact me by e-mail.', 'He had a break for the weekend, and he called me: "I am in London, so, if you want to see me, it\'s the time!"', "Research shows that 40 percent of the program's viewers are aged over 55.", "I'd guess she's about my age."]


In [7]:
print(type(dataset))
print(type(dataset['train']))
print(type(dataset['train']['level']))
print(dataset['train'][:2])


<class 'datasets.dataset_dict.DatasetDict'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'list'>
{'text': ['My mother is having her car repaired.', 'You can contact me by e-mail.'], 'level': ['B1', 'A1']}


### Preprocessing

As always, texts should be tokenized, embedded, and padded before being put into the model.  
But not to worry, there are libraries from huggingface to help with this, too.

#### Sentence processing

Different pre-trained language models may have their own preprocessing models, and that's why we should use the tokenizers trained along with that model. In our case, we are using distilBERT, so we should use the distilBERT tokenizer.  

With huggingface, loading different tokenizers is extremely easy: just import the AutoTokenizer from `transformers` and tell it what model you plan to use, and it will handle everything for you.

Reference:
 - [transformers.AutoTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoTokenizer)

In [8]:
# [ TODO ] load the distilBERT tokenizer using AutoTokenizer
from transformers import AutoTokenizer

# Download vocabulary from huggingface.co and cache.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

#tokenizer = ...

#### Label processing

Our labels also need to be processed, so let's do that next.

For this tutorial, we'll use the OneHotEncoder provided by scikit-learn.

For now, just declare a new encoder and use `fit` to learn the data. Hint: you should still end up with 6 labels.

Documents:
 - [sklearn.preprocessing.OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder)

In [9]:
tokenizer('My')

{'input_ids': [101, 2026, 102], 'attention_mask': [1, 1, 1]}

In [10]:
#X = [[dataset['train'][t]['level'], dataset['train'][t]['text']] for t in range(len(dataset['train'])) ]

In [11]:
X = dataset['train']['level']

In [12]:
X[:5]

['B1', 'A1', 'B1', 'B2', 'A1']

In [13]:
# [ TODO ] declare a new encoder and let it learn from the dataset
from sklearn.preprocessing import OneHotEncoder
import numpy as np

encoder = OneHotEncoder(handle_unknown='ignore')
X = np.array(dataset['train']['level']).reshape((-1, 1))
print(X[:3])
#print(X)
#print(type(X))
encoder.fit(X)
print(encoder.transform(X).toarray())
#encoder = ...

[['B1']
 ['A1']
 ['B1']]
[[0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [14]:
# check if you still have 6 labels

LABEL_COUNT = len(encoder.categories_[0])
print(LABEL_COUNT)

6


In [15]:
encoder.categories_

[array(['A1', 'A2', 'B1', 'B2', 'C1', 'C2'], dtype='<U2')]

In [16]:
encoder.transform(X).toarray()

array([[0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [17]:
tokenizer('My mother is having her car repaired.')

{'input_ids': [101, 2026, 2388, 2003, 2383, 2014, 2482, 13671, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

#### Process the data

To make things easier, we can write a function to process our dataset in batches. 

In [18]:

def preprocess(dataslice):
    """ Input: a batch of your dataset
        Example: { 'text': [['sentence1'], ['setence2'], ...],
                   'label': ['label1', 'label2', ...] }
    """
    
    # [ TODO ] use your tokenizor and encoder to get sentence embeddings and encoded labels

    tt = tokenizer(dataslice['text'])
    #del tt['token_type_ids']
    X = np.array(dataslice['level']).reshape((-1, 1))
    tt['label'] = encoder.transform(X).toarray()
    return  tt
    """ Output: a batch of processed dataset
        Example: { 'input_ids': ...,
                   'attention_masks': ...,
                   'label': ... }
    """

In [19]:
# map the function to the whole dataset
processed_data = dataset.map(preprocess,    # your processing function
                             batched = True # Process in batches so it can be faster
                            )

Loading cached processed dataset at /home/nlplab/yhc/.cache/huggingface/datasets/csv/data-3a5053b2d0593831/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-0a95596df1c11512.arrow
Loading cached processed dataset at /home/nlplab/yhc/.cache/huggingface/datasets/csv/data-3a5053b2d0593831/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-55e5811d7897dd15.arrow


In [20]:
print(processed_data)
print(processed_data['train'][0])
# {'text': 'My mother is having her car repaired.',
#  'level': 'B1',
#  'input_ids': [101, 2026, 2388, 2003, 2383, 2014, 2482, 13671, 1012, 102],
#  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
#  'label': [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]}


DatasetDict({
    train: Dataset({
        features: ['text', 'level', 'input_ids', 'attention_mask', 'label'],
        num_rows: 20720
    })
    test: Dataset({
        features: ['text', 'level', 'input_ids', 'attention_mask', 'label'],
        num_rows: 2300
    })
})
{'text': 'My mother is having her car repaired.', 'level': 'B1', 'input_ids': [101, 2026, 2388, 2003, 2383, 2014, 2482, 13671, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'label': [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]}


### DataCollator

You might have noticed that we skipped padding the sentences. That's because we are going to do it during training.  

To do training-time processing, we can use the DataCollator Class provided by `transformers`. And guess what - transformers has a class that will handle padding for us, too!

 - [transformers.DataCollatorWithPadding](https://huggingface.co/docs/transformers/master/en/main_classes/data_collator#transformers.DataCollatorWithPadding)

In [21]:
# [ TODO ] declare a collator to do padding during traning

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# data_collator = ...

## Training

Finally, we can move on to training.

### Preparation

We can load the pretrained model from `transformers`.  
Generally, you need to build your own model on top of BERT if you want to use BERT for some downstream tasks, but again, sequence classification is a popular topic. With the support from `transformers` library, it can be done in two lines of codes: 

1. Load `AutoModelForSequenceClassification` Class.
2. Load the pretrained model.

In [22]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,
                                                           num_labels = LABEL_COUNT)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

#### Split train/val data

The `Dataset` class we prepared before has a `train_test_split` method. You can use it to split your (processed) dataset.

Document:
 - [datasets.Dataset - Sort, shuffle, select, split, and shard](https://huggingface.co/docs/datasets/process.html#sort-shuffle-select-split-and-shard)

In [23]:
# [ TODO ] choose a validation size and split your data
train_val_dataset = processed_data['train'].train_test_split(test_size=0.1)

# train_val_dataset = ...

In [24]:
print(train_val_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'level', 'input_ids', 'attention_mask', 'label'],
        num_rows: 18648
    })
    test: Dataset({
        features: ['text', 'level', 'input_ids', 'attention_mask', 'label'],
        num_rows: 2072
    })
})


#### Setup training parameters

We are using the TrainerAPI to do the training. Trainer is yet another utility provided by huggingface, which helps you train the model with ease.  

Document:
- [transformers.TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments)
- [transformers.Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer)

In [25]:
from transformers import TrainingArguments, Trainer

In [26]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [27]:
# [ TODO ] set and tune your training properties
OUTPUT_DIR = 'model'
LEARNING_RATE = 2e-5
BATCH_SIZE =  8
EPOCH = 5

training_args = TrainingArguments(
    output_dir = OUTPUT_DIR,
    # learning_rate = LEARNING_RATE,
    per_device_train_batch_size = BATCH_SIZE,
    # per_device_eval_batch_size = BATCH_SIZE,
    num_train_epochs = EPOCH,
    # evaluation_strategy="epoch"
    # you can set more parameters here if you want
)

# now give all the information to a trainer
trainer = Trainer(
    # set your parameters here
    model = model,
    #compute_metrics=compute_metrics,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_val_dataset['train'],
    eval_dataset=train_val_dataset["test"],
    tokenizer=tokenizer
)



### Training

This is the easy part. Simply ask the trainer to train the model for you!

In [28]:
#trainer.train()

### Save for future use

Hint: try using `save_pretrained`

In [29]:
# [ TODO ] practice saving your model for future use

#model.save_pretrained('model/finetuned')

## Prediction

Now we know exactly how to train a model, but how do we use it for predicting results?

### Load finetuned model

In [30]:
# [ TODO ] load the model that you saved
from transformers import AutoConfig, AutoModelForSequenceClassification

#config = AutoConfig.from_pretrained("model/finetuned/config.json")
mymodel = AutoModelForSequenceClassification.from_pretrained('model_1/finetuned')


loading configuration file model_1/finetuned/config.json
Model config DistilBertConfig {
  "_name_or_path": "model_1/finetuned",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "multi_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file model_1/finetun

### Get the prediction

Here are a few example sentences:

In [31]:
examples = [
    # A2
    "Remember to write me a letter.",
    # B2
    "Strawberries and cream - a perfect combination.",
    "This so-called \"Perfect Evening\" was so disappointing, as well as discouraging us from coming to your Circle Theatre again.",
    # C1
    "Some may altogether give up their studies, which I think is a disastrous move.",
]

All we need to do is to transform them to embeddings, and then we can get predictions by calling your finetuned model.  

Since we don't have a DataCollator to pad the sentence and do the matrix transformation this time, we have to pad and transform the matrice on our own.

In [32]:
# Transform the sentences into embeddings
input = tokenizer(examples, truncation=True, padding=True, return_tensors="pt")
# Get the output
logits = mymodel(**input).logits
logits

tensor([[ -5.0500,   5.2882, -10.4517,  -8.7790, -10.6110, -11.2590],
        [-10.4942, -10.7707,  -8.0662,   6.3599,  -8.6863,  -7.0265],
        [-11.1577,  -9.9202,  -9.6270,  -8.8853,   8.0096,  -9.7165],
        [-12.4926, -13.5287,  -9.6069,   2.2247,  -5.6323,  -3.9449]],
       grad_fn=<AddmmBackward0>)

Logits aren't very readable for us. Let's use softmax 
activation to transform them into more probability-like numbers.

In [33]:
from torch import nn

predicts = nn.functional.softmax(logits, dim = -1)
predicts

tensor([[3.2372e-05, 9.9997e-01, 1.4595e-07, 7.7750e-07, 1.2447e-07, 6.5109e-08],
        [4.7903e-08, 3.6330e-08, 5.4301e-07, 1.0000e+00, 2.9210e-07, 1.5359e-06],
        [4.7394e-09, 1.6337e-08, 2.1904e-08, 4.5987e-08, 1.0000e+00, 2.0029e-08],
        [4.0484e-07, 1.4365e-07, 7.2526e-06, 9.9752e-01, 3.8608e-04, 2.0869e-03]],
       grad_fn=<SoftmaxBackward0>)

In [34]:
tensor = torch.argmax(predicts, dim=-1)
tensor

tensor([1, 3, 4, 3])

In [35]:
b = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']

mylist = []

for x in tensor:
    mylist.append(b[x])
a = np.array(mylist)

In [36]:
a

array(['A2', 'B2', 'C1', 'B2'], dtype='<U2')

#### Transform logits back to labels

Now you've got the output. Write a function to map it back into labels!

In [37]:
# [ TODO ] try to process the result
# (tensor([0, 4, 3, 4]), array(['A1', 'C1', 'B2', 'C1'], dtype='<U2'))

def logits_to_labels(predicts):
    a = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']
    # tensor = torch.argmax(tensor, dim=1)
    # return map(lambda x: a[x], tensor)
    tensor = torch.argmax(predicts, dim=-1)
    mylist = []
    for x in tensor:
        mylist.append(a[x])
    b = np.array(mylist)
    return b

## Evaluation

Let's see how you did!  
Load the testing data and calculate your accuracy.

We want you to calculate the three kinds of accuracy mentioned in the lecture, which will also be explained in the following section.

In [38]:
# [ TODO ] 
# load test data
import torch
# preprocess
# get predictions
test_data = processed_data['test']
input = tokenizer(test_data['text'], truncation=True, padding=True, return_tensors="pt")
logits = mymodel(**input).logits

predicts = nn.functional.softmax(logits, dim = -1)

predict_label = logits_to_labels(predicts)
# transform predictions back into labels

In [39]:
type(predict_label)

numpy.ndarray

In [40]:
#  try printing out some predictions to check if the outputs are reasonable and if you need to adjust your model at the end of every step.

for idx, (sent, level) in enumerate(zip(test_data['text'], predict_label)):
    if idx >= 10: break
    print(f'{level}: {sent}') 

C2: No longer a remote, backward, unimportant country, it became a force to be reckoned with in Europe.
B2: Unfortunately he was too fast and I couldn't keep up with him.
B2: Most mushrooms are totally harmless, but some are poisonous.
C2: This provided solid evidence that he committed the crime.
C1: You can't just accept everything you read in the newspapers at face value.
A2: Remember to write me a letter.
B1: She has long blond hair and blue eyes. She has a good figure.
B2: Nowadays the aim in clothing is not just for covering and protecting ourselves.
A2: Take two tablets, three times a day.
C2: Well, you will be if you saw our slide show and talk - members can hardly forget that relaxing afternoon when we unfolded the sails on the lake and enjoyed the tranquility of the area.


### Six Level Accuracy

Exact accuracy is probably what you're most familiar with:

$
accuracy = \frac{\#exactly\:the\:same\:levels}{\#total}
$

Example:
```
Prediction:   A1 A2 B1 B2 C1 C2
Ground truth: A2 B1 B1 B2 B2 C2
                    ^  ^     ^
```

The six level accuracy is $\frac{3}{6} = 0.5$

As the requirement, <u>your exact accuracy should be higher than $0.5$</u>.

In [44]:
# [ TODO ] calculate accuracy
correct = 0
total = 0
for idx, (sent, level, ground_truth) in enumerate(zip(test_data['text'], predict_label, test_data['level'])):
    # if idx >= 10: break
    total = total+1
    if  level == ground_truth:
        correct = correct+1
    #print(f'{level} - {ground_truth} : {sent}')
print(f'acc = {correct/total}')

acc = 0.5665217391304348


### Three Level Accuracy

Three Level Accuracy is used when you only want a more general sense of right or wrong.

$
accuracy = \frac{\#the\:same\:ABC\:levels}{\#total}
$

Example:
```
Prediction:   A1 A2 B1 B2 C1 C2
Ground truth: A2 B1 B1 B2 B2 C2
              ^     ^  ^     ^
```

The three level accuracy is $\frac{4}{6} = 0.667$

As the requirement, <u>your exact accuracy should be higher than $0.6$</u>.

In [45]:
# [ TODO ] calculate accuracy
correct = 0
total = 0
for idx, (sent, level, ground_truth) in enumerate(zip(test_data['text'], predict_label, test_data['level'])):
    #if idx >= 500: break
    total = total+1
    if level[0] == ground_truth[0]:
        correct = correct+1
    #print(f'{correct}, {level}- {ground_truth}: {sent}')
print(f'acc = {correct/total}')

acc = 0.7439130434782608


### Fuzzy accuracy

However, the level of a sentence is relatively subjective. Generally speaking, $\pm1$ errors are allowed in the real evaluation in linguistic area.  

For example, if the actual label is 'B1', we'll also consider the prediction 'right' if the model predicts 'B2' or 'A2'.

Hence, the fuzzy accuracy is

$
accuracy = \frac{\#good\:enough\:answers}{\#total}
$

Example:
```
Prediction:   0 1 2 3 4 5
Ground truth: 0 1 1 3 3 3
              ^ ^ ^ ^ ^
```

The fuzzy accuracy is $\frac{5}{6} = 0.833$

As the requirement, <u>your accuracy should be higher than $0.8$</u>.

In [52]:
# [ TODO ] calculate accuracy
correct = 0
total = 0

predict_label_num = torch.argmax(predicts, dim=1)
alph_to_num = [0, 1, 2, 3, 4, 5]
alph_to_num = {'A1':0, 'A2':1, 'B1':2, 'B2':3, 'C1':4, 'C2':5}
true_num = map(lambda x: alph_to_num[x], test_data['level'])


for idx, (sent, level, ground_truth) in enumerate(zip(test_data['text'], predict_label_num, true_num )):
    #if idx >= 30: break
    total = total+1
    if level == 0 and ground_truth ==1:
        correct = correct+1
    elif level == 5 and ground_truth == 4:
        correct = correct+1
    elif  level == ground_truth:
        correct = correct+1
    elif level == ground_truth+1 or level == ground_truth-1:
        correct = correct+1
    #print(f'{correct}, {level}- {ground_truth}: {sent}')
print(f'acc = {correct/total}')


acc = 0.8647826086956522


## TA's Note

Congratulations, you made it to the end of the tutorial! Make sure you make an appointment to show your work and turn in your finished assignment before next week's lesson. We will ask you to run your code, so double check that everything is working and that your model is saved. Don't worry if you didn't pass the evaluation requirements, you'll still get partial points for trying.

## Appendix 


<a name="Appendix-1-Install-CUDA"></a>

### Appendix 1 - Install CUDA

1. Check your GPU vs. CUDA compatibility:
   - [NVIDIA -> Your GPU Compute Capability](https://developer.nvidia.com/cuda-gpus) -> GeForce and TITAN Products
2. Check library vs. CUDA compatibility: 
   - Pytorch: [Previous PyTorch Versions](https://pytorch.org/get-started/previous-versions/)
   - Tensorflow: [Linux/MacOX](https://www.tensorflow.org/install/source#tested_build_configurations) or [Windows](https://www.tensorflow.org/install/source_windows#tested_build_configurations)
3. Note the highest CUDA version that fits your system.

#### >> for conda/mamba users

You can directly install CUDA library with the selected CUDA version.
1. Get [the driver for NVIDIA GPU](https://www.nvidia.com/download/index.aspx)
2. `conda/mamba install -c conda-forge cudatoolkit=${VERSION}`

#### >> for non-conda users

1. Get [the driver for NVIDIA GPU](https://www.nvidia.com/download/index.aspx)
2. Download and install [CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit-archive)
3. Download and install [cuDNN Library](https://developer.nvidia.com/rdp/cudnn-archive)

### Appendix 2 - Further Readings

1. [Huggingface Official Tutorials](https://github.com/huggingface/notebooks/tree/master/examples)
2. How to use Bert with other downstream tasks: [How to use BERT from the Hugging Face transformer library](https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209): 
3. Training with pytorch backend: [transformers-tutorials](https://github.com/abhimishra91/transformers-tutorials)
4. A more complicated example that include manual data/training processing with Pytorch: [Transformers for Multi-Label Classification made simple](https://towardsdatascience.com/transformers-for-multilabel-classification-71a1a0daf5e1)
5. [Text Classification with tensorflow](https://github.com/huggingface/notebooks/blob/master/examples/text_classification-tf.ipynb): tensorflow example